# Step 0: Use GPU and update cudnn to 8.1.0

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

!pip install googledrivedownloader

gdd.download_file_from_google_drive(
    file_id='1BfjF8Vv6jJ626i5IUaul3-ZBM010-bxC',
    dest_path='./libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb'
)

!dpkg -i "libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb"

!ls -l /usr/lib/x86_64-linux-gnu/libcudnn.so.8*


(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb ...
Unpacking libcudnn8 (8.1.0.77-1+cuda11.2) over (8.0.5.39-1+cuda11.1) ...
Setting up libcudnn8 (8.1.0.77-1+cuda11.2) ...
Processing triggers for libc-bin (2.27-3ubuntu1.3) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

lrwxrwxrwx 1 root root     17 Jan 24  2021 /usr/lib/x86_64-linux-gnu/libcudnn.so.8 -> libcudnn.so.8.1.0
-rw-r--r-- 1 root root 158264 Jan 24  2021 /usr/lib/x86_64-linux-gnu/libcudnn.so.8.1.0


# Step 1: Git clone your project

In [ ]:
!git clone https://github.com/gyes00205/VRDL_HW2.git

Cloning into 'VRDL_HW2'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 14 (delta 2), reused 12 (delta 0), pack-reused 0
Unpacking objects: 100% (14/14), done.


# Step 2: Install your requirements

In [ ]:
%cd VRDL_HW2

# Git clone TF-object-dectection-API
!git clone https://github.com/tensorflow/models.git

%cd models/research/

!protoc object_detection/protos/*.proto --python_out=.

!cp object_detection/packages/tf2/setup.py ./

!python -m pip install --use-deprecated=legacy-resolver .

In [ ]:
import os
import cv2
import time
from tqdm import tqdm
os.environ['PYTHONPATH'] += ':/content/VRDL_HW2/models/research/:\
    /content/VRDL_HW2//models/research/slim'

%cd ../..


/content/VRDL_HW2


# Step 3: Wget testing data

In [ ]:
# 1. Download the testing data

gdd.download_file_from_google_drive(
    file_id='1Fm-avdeNgzhPxhvia0iw9yZzcoOggy7I',
    dest_path='./test.zip',
    unzip=True
)

# 2. Read the image_name and put them into the list
# You need to modify the path to fit your test_folder
data_listdir = os.listdir("/content/VRDL_HW2/test")


Unzipping...Done.


# Step 4: Run inference and bench mark

In [ ]:
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

gdd.download_file_from_google_drive(
    file_id='1QjFedhxrj4p4lF6jpUYZYNLa0rcJ4Y5y',
    dest_path='./efficientdet_d2_ckpt202.zip',
    unzip=True
)

# 1. Load your model and weights
label_map = '/content/VRDL_HW2/data/label_map.pbtxt'
checkpoint = "/content/VRDL_HW2/efficientdet_d2_ckpt202/saved_model"

# 2. Initialize the model
model = tf.saved_model.load(checkpoint)


Unzipping...Done.


In [ ]:
# Test your inference time
TEST_IMAGE_NUMBER = 100  # This number is fixed.
test_img_list = []

# Read image (Be careful with the image order)
data_listdir.sort(key=lambda x: int(x[:-4]))
for img_name in data_listdir[:TEST_IMAGE_NUMBER]:
    img_path = os.path.join("/content/VRDL_HW2/test", img_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    input_tensor = tf.convert_to_tensor(img)
    input_tensor = input_tensor[tf.newaxis, ...]
    test_img_list.append(input_tensor)

start_time = time.time()
for img in tqdm(test_img_list):
    # your model prediction
    pred = model(img)

end_time = time.time()
print(
    "\nInference time per image: ",
    (end_time - start_time) / len(test_img_list)
)

# Remember to screenshot!


100%|██████████| 100/100 [00:18<00:00,  5.29it/s]


Inference time per image:  0.18901926279067993


# STEP 5: Generate answer.json for submission on Codalab
The answer.json has the same format as [COCO dataset results](https://cocodataset.org/#format-results)

In [ ]:
!python detect.py \
--saved_model_path=efficientdet_d2_ckpt202/ \
--test_path=test \
--output_path=output_image \
--min_score_thresh=0.0 \
--label_map=data/label_map.pbtxt
